In [1]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

In [2]:
from tensorflow.keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Softmax, Dense, Lambda, BatchNormalization, ReLU
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers.legacy import SGD
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
# Convert y_train into one-hot format
temp = []
for i in range(len(y_train)):
    temp.append(to_categorical(y_train[i], num_classes=10))
y_train = np.array(temp)
# Convert y_test into one-hot format
temp = []
for i in range(len(y_test)):    
    temp.append(to_categorical(y_test[i], num_classes=10))
y_test = np.array(temp)

In [5]:
#reshaping
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

#normalizing
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

In [6]:
inputs = Input(shape=(28,28,1))
out = Conv2D(4, 3, use_bias=True)(inputs)
out = BatchNormalization()(out)
out = ReLU()(out)
out = AveragePooling2D()(out)
out = Conv2D(8, 3, use_bias=True)(out)
out = BatchNormalization()(out)
out = ReLU()(out)
out = AveragePooling2D()(out)
out = Flatten()(out)
out = Dense(10, activation=None)(out)
out = Softmax()(out)
model = Model(inputs, out)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 4)         40        
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 4)        16        
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 26, 26, 4)         0         
                                                                 
 average_pooling2d (AverageP  (None, 13, 13, 4)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 8)         296   

In [8]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(learning_rate=0.01, momentum=0.9),
    metrics=['acc']
    )

In [9]:
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/15
   1/1875 [..............................] - ETA: 5:16 - loss: 2.6515 - acc: 0.0625

2023-10-24 02:23:16.557845: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1875/1875 [==============================] - 8s 4ms/step - loss: 0.1896 - acc: 0.9434 - val_loss: 0.0808 - val_acc: 0.9753
Epoch 2/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0787 - acc: 0.9761 - val_loss: 0.0602 - val_acc: 0.9808
Epoch 3/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0650 - acc: 0.9803 - val_loss: 0.0546 - val_acc: 0.9831
Epoch 4/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0592 - acc: 0.9816 - val_loss: 0.0613 - val_acc: 0.9812
Epoch 5/15
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0536 - acc: 0.9837 - val_loss: 0.0594 - val_acc: 0.9808
Epoch 6/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0497 - acc: 0.9847 - val_loss: 0.0618 - val_acc: 0.9794
Epoch 7/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0470 - acc: 0.9855 - val_loss: 0.0824 - val_acc: 0.9748
Epoch 8/15
1875/1875 [==============================] - 8s 4ms/step - los

In [10]:
X = X_test[0]
X.shape, X.min(), X.max()

((28, 28, 1), 0.0, 1.0)

In [11]:
model.predict(X.reshape(1,28,28,1)).argmax()

1/1 [==============================] - 0s 45ms/step


7

In [12]:
def Conv2DInt(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    Input = [[[str(input[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    Weights = [[[[str(weights[i][j][k][l] % p) for l in range(nFilters)] for k in range(nChannels)] for j in range(kernelSize)] for i in range(kernelSize)]
    Bias = [str(bias[i] % p) for i in range(nFilters)]
    out = [[[0 for _ in range(nFilters)] for _ in range((nCols - kernelSize)//strides + 1)] for _ in range((nRows - kernelSize)//strides + 1)]
    remainder = [[[None for _ in range(nFilters)] for _ in range((nCols - kernelSize)//strides + 1)] for _ in range((nRows - kernelSize)//strides + 1)]
    for i in range((nRows - kernelSize)//strides + 1):
        for j in range((nCols - kernelSize)//strides + 1):
            for m in range(nFilters):
                for k in range(nChannels):
                    for x in range(kernelSize):
                        for y in range(kernelSize):
                            out[i][j][m] += input[i*strides+x][j*strides+y][k] * weights[x][y][k][m]
                out[i][j][m] += bias[m]
                remainder[i][j][m] = str(out[i][j][m] % n)
                out[i][j][m] = str(out[i][j][m] // n % p)
    return Input, Weights, Bias, out, remainder

In [13]:
X_in = [[[int(X[i][j][0]*1e18)] for j in range(28)] for i in range(28)]
conv2d_1_weights = [[[[int(model.layers[1].weights[0][i][j][k][l]*1e18) for l in range(4)] for k in range(1)] for j in range(3)] for i in range(3)]
conv2d_1_bias = [int(model.layers[1].weights[1][i]*1e36) for i in range(4)]

In [14]:
X_in, conv2d_1_weights, conv2d_1_bias, conv2d_1_out, conv2d_1_remainder = Conv2DInt(28, 28, 1, 4, 3, 1, 10**18, X_in, conv2d_1_weights, conv2d_1_bias)
conv2d_1_out[0][0]

['21888242871839275222246405745257275088548364400416034343698204178650554445275',
 '21888242871839275222246405745257275088548364400416034343698203817637026120037',
 '21888242871839275222246405745257275088548364400416034343698204118313744391231',
 '48458611567652']

In [15]:
conv2d_model = Model(inputs, model.layers[1].output)
conv2d_model.predict(X.reshape(1,28,28,1))[0][0][0]

1/1 [==============================] - 0s 18ms/step


array([-7.9252541e-06, -3.6893881e-04, -6.8262067e-05,  4.8458613e-05],
      dtype=float32)

In [16]:
gamma = model.layers[2].weights[0].numpy()
beta = model.layers[2].weights[1].numpy()
moving_mean = model.layers[2].weights[2].numpy()
moving_var = model.layers[2].weights[3].numpy()
epsilon = model.layers[2].epsilon

In [17]:
a1 = gamma/(moving_var+epsilon)**.5
b1 = beta-gamma*moving_mean/(moving_var+epsilon)**.5
a1, b1

(array([2.841136 , 2.7350745, 2.0774858, 1.246628 ], dtype=float32),
 array([ 0.9730277 , -0.0507403 , -0.07507665, -0.5589396 ], dtype=float32))

In [18]:
def BatchNormalizationInt(nRows, nCols, nChannels, n, X_in, a_in, b_in):
    X = [[[str(X_in[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    A = [str(a_in[k] % p) for k in range(nChannels)]
    B = [str(b_in[k] % p) for k in range(nChannels)]
    out = [[[None for _ in range(nChannels)] for _ in range(nCols)] for _ in range(nRows)]
    remainder = [[[None for _ in range(nChannels)] for _ in range(nCols)] for _ in range(nRows)]
    for i in range(nRows):
        for j in range(nCols):
            for k in range(nChannels):
                out[i][j][k] = (X_in[i][j][k]*a_in[k] + b_in[k])
                remainder[i][j][k] = str(out[i][j][k] % n)
                out[i][j][k] = str(out[i][j][k] // n % p)
    return X, A, B, out, remainder

In [19]:
bn_1_in = [[[int(conv2d_1_out[i][j][k]) if int(conv2d_1_out[i][j][k]) < p//2 else int(conv2d_1_out[i][j][k]) - p for k in range(4)] for j in range(26)] for i in range(26)]
bn_1_a = [int(a1[i]*1e18) for i in range(4)]
bn_1_b = [int(b1[i]*1e36) for i in range(4)]

In [20]:
_, bn_1_a, bn_1_b, bn_1_out, bn_1_remainder = BatchNormalizationInt(26, 26, 4, 10**18, bn_1_in, bn_1_a, bn_1_b)
bn_1_out[0][0]

['973005189421817592',
 '21888242871839275222246405745257275088548364400416034343698152437199136300055',
 '21888242871839275222246405745257275088548364400416034343698128968107786241045',
 '21888242871839275222246405745257275088548364400416034343697645307409523760974']

In [21]:
bn_1_model = Model(inputs, model.layers[2].output)
bn_1_model.predict(X.reshape(1,28,28,1))[0][0][0]

1/1 [==============================] - 0s 22ms/step


array([ 0.97300524, -0.05174941, -0.07521845, -0.55887914], dtype=float32)

In [22]:
relu_1_in = [[[int(bn_1_out[i][j][k]) for k in range(4)] for j in range(26)] for i in range(26)]
relu_1_out = [[[str(relu_1_in[i][j][k]) if relu_1_in[i][j][k] < p//2 else 0 for k in range(4)] for j in range(26)] for i in range(26)]

In [23]:
avg2d_1_in = [[[int(relu_1_out[i][j][k]) for k in range(4)] for j in range(26)] for i in range(26)]

In [24]:
def AveragePooling2DInt (nRows, nCols, nChannels, poolSize, strides, input):
    Input = [[[str(input[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    out = [[[0 for _ in range(nChannels)] for _ in range((nCols-poolSize)//strides + 1)] for _ in range((nRows-poolSize)//strides + 1)]
    remainder = [[[None for _ in range(nChannels)] for _ in range((nCols-poolSize)//strides + 1)] for _ in range((nRows-poolSize)//strides + 1)]
    for i in range((nRows-poolSize)//strides + 1):
        for j in range((nCols-poolSize)//strides + 1):
            for k in range(nChannels):
                for x in range(poolSize):
                    for y in range(poolSize):
                        out[i][j][k] += input[i*strides+x][j*strides+y][k]
                remainder[i][j][k] = str(out[i][j][k] % poolSize**2 % p)
                out[i][j][k] = str(out[i][j][k] // poolSize**2 % p)
    return Input, out, remainder

In [25]:
_, avg2d_1_out, avg2d_1_remainder = AveragePooling2DInt(26, 26, 4, 2, 2, avg2d_1_in)
avg2d_1_out[5][6]

['1312195747641412224', '17351024717876988', '381448215010339593', '0']

In [26]:
avg2d_1_model = Model(inputs, model.layers[4].output)
avg2d_1_model.predict(X.reshape(1,28,28,1))[0][5][6]

1/1 [==============================] - 0s 28ms/step


array([1.3121958 , 0.01735102, 0.38144833, 0.        ], dtype=float32)

In [27]:
conv2d_2_in = [[[int(avg2d_1_out[i][j][k]) for k in range(4)] for j in range(13)] for i in range(13)]
conv2d_2_weights = [[[[int(model.layers[5].weights[0][i][j][k][l]*1e18) for l in range(8)] for k in range(4)] for j in range(3)] for i in range(3)]
conv2d_2_bias = [int(model.layers[5].weights[1][i]*1e36) for i in range(8)]

In [28]:
_, conv2d_2_weights, conv2d_2_bias, conv2d_2_out, conv2d_2_remainder = Conv2DInt(13, 13, 4, 8, 3, 1, 10**18, conv2d_2_in, conv2d_2_weights, conv2d_2_bias)
conv2d_2_out[0][0]

['151443532606342120',
 '21888242871839275222246405745257275088548364400416034343695445103896159586204',
 '1368543458414900467',
 '21888242871839275222246405745257275088548364400416034343697889320797389307844',
 '21888242871839275222246405745257275088548364400416034343696909822999783625702',
 '3064925807006993607',
 '273553711551724155',
 '21888242871839275222246405745257275088548364400416034343697690313879935243528']

In [29]:
conv2d_2_model = Model(inputs, model.layers[5].output)
conv2d_2_model.predict(X.reshape(1,28,28,1))[0][0][0]

1/1 [==============================] - 0s 27ms/step


array([ 0.15144362, -2.7590828 ,  1.3685436 , -0.3148657 , -1.2943636 ,
        3.064926  ,  0.27355385, -0.5138727 ], dtype=float32)

In [30]:
gamma = model.layers[6].weights[0].numpy()
beta = model.layers[6].weights[1].numpy()
moving_mean = model.layers[6].weights[2].numpy()
moving_var = model.layers[6].weights[3].numpy()
epsilon = model.layers[6].epsilon

In [31]:
a2 = gamma/(moving_var+epsilon)**.5
b2 = beta-gamma*moving_mean/(moving_var+epsilon)**.5
a2, b2

(array([1.4172864, 1.0896717, 1.2455306, 1.9744203, 1.5216775, 1.6048892,
        1.5560555, 1.5188278], dtype=float32),
 array([ 0.9693597 ,  2.5460322 , -2.3216164 ,  1.1771976 ,  1.7650728 ,
        -5.5845942 , -0.36191303,  0.58835894], dtype=float32))

In [32]:
bn_2_in = [[[int(conv2d_2_out[i][j][k]) if int(conv2d_2_out[i][j][k]) < p//2 else int(conv2d_2_out[i][j][k]) - p for k in range(8)] for j in range(11)] for i in range(11)]
bn_2_a = [int(a2[i]*1e18) for i in range(8)]
bn_2_b = [int(b2[i]*1e36) for i in range(8)]

In [33]:
_, bn_2_a, bn_2_b, bn_2_out, bn_2_remainder = BatchNormalizationInt(11, 11, 8, 10**18, bn_2_in, bn_2_a, bn_2_b)
bn_2_out[0][0]

['1183998554440588778',
 '21888242871839275222246405745257275088548364400416034343697743724366639529790',
 '21888242871839275222246405745257275088548364400416034343697587132926760373824',
 '555520188028190246',
 '21888242871839275222246405745257275088548364400416034343697999655475625376512',
 '21888242871839275222246405745257275088548364400416034343697538458512894177905',
 '63751744556936369',
 '21888242871839275222246405745257275088548364400416034343698012061380413810399']

In [34]:
bn_2_model = Model(inputs, model.layers[6].output)
bn_2_model.predict(X.reshape(1,28,28,1))[0][0][0]

1/1 [==============================] - 0s 32ms/step


array([ 1.1839986 , -0.46046233, -0.61705345,  0.5555204 , -0.20453101,
       -0.66572803,  0.06375193, -0.1921252 ], dtype=float32)

In [35]:
relu_2_in = [[[int(bn_2_out[i][j][k]) for k in range(8)] for j in range(11)] for i in range(11)]
relu_2_out = [[[str(relu_2_in[i][j][k]) if relu_2_in[i][j][k] < p//2 else 0 for k in range(8)] for j in range(11)] for i in range(11)]
relu_2_out[0][0]

['1183998554440588778',
 0,
 0,
 '555520188028190246',
 0,
 0,
 '63751744556936369',
 0]

In [36]:
relu_2_model = Model(inputs, model.layers[7].output)
relu_2_model.predict(X.reshape(1,28,28,1))[0][0][0]

1/1 [==============================] - 0s 30ms/step


array([1.1839986 , 0.        , 0.        , 0.5555204 , 0.        ,
       0.        , 0.06375193, 0.        ], dtype=float32)

In [37]:
avg2d_2_in = [[[int(relu_2_out[i][j][k]) if int(relu_2_out[i][j][k]) < p//2 else int(relu_2_out[i][j][k]) - p for k in range(8)] for j in range(11)] for i in range(11)]

In [38]:
_, avg2d_2_out, avg2d_2_remainder = AveragePooling2DInt(11, 11, 8, 2, 2, avg2d_2_in)
avg2d_2_out[3][3]

['0',
 '3041275199357812815',
 '880404200542187368',
 '751626574290071696',
 '4631629684299696339',
 '0',
 '0',
 '141002623674408652']

In [39]:
avg2d_2_model = Model(inputs, model.layers[8].output)
avg2d_2_model.predict(X.reshape(1,28,28,1))[0][3][3]

1/1 [==============================] - 0s 30ms/step


array([0.        , 3.0412755 , 0.88040376, 0.7516271 , 4.6316295 ,
       0.        , 0.        , 0.14100271], dtype=float32)

In [40]:
flatten_out = [avg2d_2_out[i][j][k] for i in range(5) for j in range(5) for k in range(8)]
flatten_out[100:120]

['240465720017978049',
 '312962931075997403',
 '16422062895818568',
 '0',
 '0',
 '1115397662044723147',
 '3743826354975568930',
 '1282135426254877774',
 '2558492900397241085',
 '0',
 '69224497325539673',
 '0',
 '60798814494206494',
 '2302583886918782205',
 '379552091654971946',
 '0',
 '1609754191355983350',
 '0',
 '27599924897036794',
 '30247813336316648']

In [41]:
flatten_model = Model(inputs, model.layers[9].output)
flatten_model.predict(X.reshape(1,28,28,1))[0][100:120]

1/1 [==============================] - 0s 32ms/step


array([0.24046564, 0.312963  , 0.0164221 , 0.        , 0.        ,
       1.1153977 , 3.7438262 , 1.2821352 , 2.5584927 , 0.        ,
       0.0692246 , 0.        , 0.06079884, 2.3025842 , 0.37955213,
       0.        , 1.6097541 , 0.        , 0.02759986, 0.03024786],
      dtype=float32)

In [42]:
dense_in = [int(flatten_out[i]) if int(flatten_out[i]) < p//2 else int(flatten_out[i]) - p for i in range(200)]
dense_weights = [[int(model.layers[10].weights[0][i][j]*1e18) for j in range(10)] for i in range(200)]
dense_bias = [int(model.layers[10].weights[1][i]*1e36) for i in range(10)]

In [43]:
def DenseInt(nInputs, nOutputs, n, input, weights, bias):
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]
    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += input[i] * weights[i][j]
        out[j] += bias[j]
        remainder[j] = str(out[j] % n)
        out[j] = str(out[j] // n % p)
    return Input, Weights, Bias, out, remainder

In [44]:
_, dense_weights, dense_bias, dense_out, dense_remainder = DenseInt(200, 10, 10**18, dense_in, dense_weights, dense_bias)
dense_out

['21888242871839275222246405745257275088548364400416034343696001558406187208579',
 '21888242871839275222246405745257275088548364400416034343694494011253998843463',
 '2502586410316628302',
 '7723360444146681933',
 '21888242871839275222246405745257275088548364400416034343688179933383346961393',
 '21888242871839275222246405745257275088548364400416034343697101907583287035462',
 '21888242871839275222246405745257275088548364400416034343680804147065276585857',
 '21047995401855287971',
 '21888242871839275222246405745257275088548364400416034343695951614145619839501',
 '4230622081176419220']

In [45]:
dense_out

['21888242871839275222246405745257275088548364400416034343696001558406187208579',
 '21888242871839275222246405745257275088548364400416034343694494011253998843463',
 '2502586410316628302',
 '7723360444146681933',
 '21888242871839275222246405745257275088548364400416034343688179933383346961393',
 '21888242871839275222246405745257275088548364400416034343697101907583287035462',
 '21888242871839275222246405745257275088548364400416034343680804147065276585857',
 '21047995401855287971',
 '21888242871839275222246405745257275088548364400416034343695951614145619839501',
 '4230622081176419220']

In [46]:
dense_model = Model(inputs, model.layers[-2].output)
dense_model.predict(X.reshape(1,28,28,1))[0]

1/1 [==============================] - 0s 35ms/step


array([ -2.2026286,  -3.7101758,   2.5025864,   7.7233634, -10.024254 ,
        -1.1022782, -17.40004  ,  21.047997 ,  -2.2525737,   4.230623 ],
      dtype=float32)

In [47]:
in_json = {
    "in": X_in,
    "conv2d_1_weights": conv2d_1_weights,
    "conv2d_1_bias": conv2d_1_bias,
    "conv2d_1_out": conv2d_1_out,
    "conv2d_1_remainder": conv2d_1_remainder,
    "bn_1_a": bn_1_a,
    "bn_1_b": bn_1_b,
    "bn_1_out": bn_1_out,
    "bn_1_remainder": bn_1_remainder,
    "relu_1_out": relu_1_out,
    "avg2d_1_out": avg2d_1_out,
    "avg2d_1_remainder": avg2d_1_remainder,
    "conv2d_2_weights": conv2d_2_weights,
    "conv2d_2_bias": conv2d_2_bias,
    "conv2d_2_out": conv2d_2_out,
    "conv2d_2_remainder": conv2d_2_remainder,
    "bn_2_a": bn_2_a,
    "bn_2_b": bn_2_b,
    "bn_2_out": bn_2_out,
    "bn_2_remainder": bn_2_remainder,
    "relu_2_out": relu_2_out,
    "avg2d_2_out": avg2d_2_out,
    "avg2d_2_remainder": avg2d_2_remainder,
    "flatten_out": flatten_out,
    "dense_weights": dense_weights,
    "dense_bias": dense_bias,
    "dense_out": dense_out,
    "dense_remainder": dense_remainder,
    "argmax_out": "7"
}

In [48]:
import json

In [49]:
with open("mnist_input.json", "w") as f:
    json.dump(in_json, f)